In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/drlcourse-main/day1/tabular_mdp
%ls

/content/drive/MyDrive/Colab Notebooks/drlcourse-main/day1/tabular_mdp
day1_tabular_mdp.pdf  gridworld.py  pendulum.py  tabular.ipynb
gridworld.png         gym_test.py   solvers.py   utils.py


In [ ]:
import numpy as np
import time
from utils import *

# 1. Prerequisites

## 1.1 Toy Example

In [ ]:
R = np.array([[-2.0, -0.5],
              [-1.0, -3.0]])

P = np.array([[0.75, 0.25],
              [0.75, 0.25],
              [0.25, 0.75],
              [0.25, 0.75]])

gamma = 0.9

## 1.2 Define Q-function and Greedy policy
First define Q-function from the value function: 
$$Q(s, a) = r(s, a) + \gamma \sum_{s^\prime} p(s^\prime \mid s, a) v(s^\prime)$$

Greedy policy from Q-function: $\pi_{\text{greedy}}(s) = \arg\max_a Q(s, a)$.

In [ ]:
def q_ftn(P, R, gamma, v):
    """
    given v, get corresponding q
    hint : v's shape = (|S|, 1)
           P's shape = (|S| * |A|, |S|)
           R's shape = (|S|, |A|)
           [Fill here]'s shape = (|S| * |A|, 1)
           numpy.matmul(A, x) returns a matrix multiplication result A * x.
    """
    # TODO : Complete the following line.
    return R + gamma * np.reshape([Fill here], newshape=R.shape, order='F')

def greedy(P, R, gamma, v):
    """
    construct greedy policy by pi(s) = argmax_a q(s, a)
    """
    # TODO : Complete the below line.
    q = q_ftn(P, R, gamma, v)
    pi = np.argmax(q, axis=[Fill here])

    return pi

# 2. Value Iteration

Value iteration iteratively applies the *Bellman Optimality Operator*

\begin{equation*}
\mathcal{T}v\,(s) := \max_{a \in \mathcal{A}}\bigg[ r\,(s,a) + \gamma \sum_{s'\in\mathcal{S}} p\,(s'|s,a)\;v\,(s') \bigg]
\end{equation*}

In [ ]:
def bellman_update(P, R, gamma, v):
    q = q_ftn(P, R, gamma, v)
    # TODO : complete the following line
    v_next = np.max(q, axis=1, keepdims=True)
    return v_next

From calculated $v^*$, we can calculate $\pi^*$ by Q-function

$$ \pi^* := \arg\max_{a\in\mathcal{A}} Q^*(s,a) $$

In [ ]:
EPS = 1e-6
nS, nA = R.shape
# Initialize v
v = np.zeros(shape=(nS, 1), dtype=np.float)

# Value Iteration
count = 0
start = time.time()
while True:
    v_next = bellman_update(P, R, gamma, v)
    if np.linalg.norm(v_next - v, ord=np.inf) < EPS:
        break
    v = v_next
    count += 1
print('Iteration terminated in {} steps, Ellapsed time {:4f} sec\n'.format(count, time.time() - start))

# TODO: construct policy obtained v^*
pi = greedy(P, R, gamma, v)
print_result(v, pi)

Iteration terminated in 129 steps, Ellapsed time 0.004033 sec

+========== Result ==========+
optimal value function : 
v(s1) = -7.327576823826019
v(s2) = -7.672404410032915
optimal policy : 
pi(s1) = a2
pi(s2) = a1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


# 2. Policy Iteration

Policy Iteration iterates between these 2 steps


*   Policy Evaluation : calculate policy value function $v^\pi$
*   Policy Update : from calculated $v^\pi$, update $\pi$

## 2.1 Policy Evaluation: Get induced dynamics

Let, start from getting $\hat{P}$ and $\hat{R}$

Suppose 2 states, 2 actions, and $\pi = [a_1, a_0]$

We should select row vector of $P$
\begin{equation*}
p(\cdot|s_0,a_0)\\
p(\cdot|s_1,a_0) \rightarrow p(\cdot|s_1,\pi(s_1))\\
p(\cdot|s_0,a_1) \rightarrow p(\cdot|s_0,\pi(s_0))\\
p(\cdot|s_1,a_1)\\
\end{equation*}

So, $P^\pi$ is a rearranged submatrix of $P$

$$P^{\pi} = P\;[\;[2,1], \; : \;]$$

Note that row number can be obtained via:

$$2 = 0(s_0) + 2 \cdot 1(\pi(s_0))$$

$$1 = 1(s_1) + 2 \cdot 0(\pi(s_1))$$

In [ ]:
def induced_dynamic(nS, P, R, pi):
    """
    given policy pi, compute induced dynamic P^pi & R^pi
    """
    # TODO : complete the below line
    rows = [Fill here]
    P_pi = P[rows]
    R_pi = np.array([[R[s, pi[s]]] for s in range(nS)])

    return P_pi, R_pi

## 2.2 Policy Evaluation: Calculate $v^\pi$
With calculated induced dynamics, we can compute $v^\pi$


\begin{equation*}
v^\pi = (I - \gamma P^\pi)^{-1} R^{\pi}
\end{equation*}


To solve the above equation, you may use ```numpy.linalg.solve(A, b)```.
This returns a solution of the linear equations $A x = b$, i.e., $x = A^{-1}b$.

Note that $v^\pi$ can be attained with iterating the Bellman operator $\mathcal{T}^\pi$

$$\mathcal{T}^\pi v := R^\pi +\gamma P^\pi v^\pi $$

In [ ]:
def eval_policy(nS, P, R, gamma, pi):
    """
    policy evaluation
    """
    P_pi, R_pi = induced_dynamic(nS, P, R, pi)

    Id = np.identity(nS)
    # discounted reward problem
    # TODO : Complete the below lines. (Hint. numpy.linalg.solve(A, b) returns a solution of A * x = b., i.e. A^{-1}b)
    A = 
    b = 
    v_pi = np.linalg.solve(A, b)
    
    return v_pi

## 2.2 Policy Update

Based on Q-function from $v^\pi$, let's update our policy

\begin{equation*}
\pi'(s) = \arg\max_{a \in \mathcal{A}} Q^\pi(s,a)
\end{equation*}

## 2.3 Policy Iteration Loop

In [ ]:
nS, nA = R.shape

# initialize policy
pi = np.random.randint(nA, size=nS)

count = 0
start = time.time()
while True:
    v = eval_policy(nS, P, R, gamma, pi)
    # TODO: update policy
    pi_next = greedy(P, R, gamma, v)
    if (pi_next == pi).all():
        break
    pi = pi_next
    count += 1
print('Iteration terminated in {} steps, Ellapsed time {:4f} sec\n'.format(count, time.time() - start))

print_result(v, pi)

Iteration terminated in 1 steps, Ellapsed time 0.000902 sec

+========== Result ==========+
optimal value function : 
v(s1) = -7.327586206896552
v(s2) = -7.6724137931034475
optimal policy : 
pi(s1) = a2
pi(s2) = a1


# 3. Example : [Taxi-v3](https://gym.openai.com/envs/Taxi-v3/) from OpenAI Gym

## 3.2 Load model from Gym!

In [ ]:
def GetDynamics(env, check_deterministic=False):
    env.P[env.nS] = {a:[(1.0, env.nS, 0, False)] for a in range(env.nA)}
    
    P = np.zeros(((env.nS + 1) * env.nA, env.nS + 1))
    R = np.zeros((env.nS + 1, env.nA))

    for row_idx in range( (env.nS + 1) * env.nA ):
        s = row_idx % (env.nS + 1)
        a = row_idx // (env.nS + 1)
        for (prob, new_state, reward, done) in env.P[s][a]:
            if check_deterministic:
                assert (prob == 1 or prob == 0)
            if done:
                P[row_idx][env.nS] = prob
            else:
                P[row_idx][new_state] = prob
            R[s][a] = reward

    return P, R

In [ ]:
import gym
env = gym.make('Taxi-v3')

print("Environment size - (|S|, |A|) = (", env.nS, ",", env.nA, ")")
P_taxi, R_taxi = GetDynamics(env)

Environment size - (|S|, |A|) = ( 500 , 6 )


## 3.2 VI and PI

In [ ]:
# VI
nS, nA = R_taxi.shape
EPS = 1e-6

# Initialize v
v = np.zeros(shape=(nS, 1), dtype=np.float)

# Value Iteration
count = 0
start = time.time()
while True:
    v_next = bellman_update(P_taxi, R_taxi, gamma, v)
    if np.linalg.norm(v_next - v, ord=np.inf) < EPS:
        break
    v = v_next
    count += 1
print('Iteration terminated in {} steps, Ellapsed time {:4f} sec\n'.format(count, time.time() - start))

pi = greedy(P_taxi, R_taxi, gamma, v)

Iteration terminated in 18 steps, Ellapsed time 0.017905 sec



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
# PI
nS, nA = R_taxi.shape

# initialize policy
pi = np.random.randint(nA, size=nS)

count = 0
start = time.time()
while True:
    v = eval_policy(nS, P_taxi, R_taxi, gamma, pi)
    # TODO: update policy
    pi_next = greedy(P_taxi, R_taxi, gamma, v)
    if (pi_next == pi).all():
        break
    pi = pi_next
    count += 1
print('Iteration terminated in {} steps, Ellapsed time {:4f} sec\n'.format(count, time.time() - start))

Iteration terminated in 16 steps, Ellapsed time 0.201125 sec



## 3.3 Simulate your policy with Gym!

In [ ]:
# Initialize environment
state = env.reset()

for t in range(1000):
    print('+=======  Stage {}  =======+\n'.format(t))
    env.render()

    action = pi[state]
    
    # Simulate 1 step
    state, reward, done, info = env.step(action) 
    print('')
   
    # done is used to check terminal condition    
    if done:
      env.render()
      print("\n+=======  Success!   =======+")
      break

# Close environment
env.close()

+=======  Stage 0  =======+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


+=======  Stage 1  =======+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

+=======  Stage 2  =======+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

+=======  Stage 3  =======+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

+=======  Stage 4  =======+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+=======  Stage 5  =======+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+=======  Stage 6  =======+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

+=======  Stage 7  =======+

+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

+=======  Stage 8  